# Tool Development Best Practices: Interactive Jupyter Notebook

Welcome! This notebook is designed to teach key concepts in AI tool development. The examples and use cases are based on principles from Chip Huyen's 2025 AI Engineering framework and leverage best practices from the AI Engineering resource.

## Learning Goals
- Understand foundational best practices in tool development
- Create a tool
- use a tool

In [ ]:
pip install litellm

In [ ]:
# Setup
from litellm import completion
import litellm
import json

# model="ollama/driaforall/tiny-agent-a:0.5b"
model="ollama/llama3.1:8b"

assert litellm.supports_function_calling(model=model) == True


### Creating a Tool for LLM Function Calling

Let's create a unit conversion tool that the LLM can use via function calling:

In [ ]:
# Define a tool for unit conversion
unit_conversion_tool = {
    "type": "function",
    "function": {
        "name": "convert_units",
        "description": "Convert between different units of measurement. Use shortened measurement names eg, pounds -> lbs; kilograms -> kg",
        "parameters": {
            "type": "object",
            "properties": {
                "value": {
                    "type": "number",
                    "description": "The value to convert"
                },
                "from_unit": {
                    "type": "string",
                    "description": "The source unit. Must be one of: (lbs', 'kg', 'miles', 'km')"
                },
                "to_unit": {
                    "type": "string",
                    "description": "The target unit. Must be one of: ('lbs', 'kg', 'miles', 'km')"
                }
            },
            "required": ["value", "from_unit", "to_unit"]
        }
    }
}

# Function to handle the unit conversion
def convert_units(value, from_unit, to_unit):
    """Convert between different units of measurement with alias support"""
    value = float(value)
    # Define aliases for units
    aliases = {
        "pound": "lbs",
        "pounds": "lbs",
        "lb": "lbs",
        "kilogram": "kg",
        "kilograms": "kg",
        "mile": "miles",
        "miles": "miles",
        "km": "km",
        "kilometer": "km",
        "kilometers": "km"
    }

    # Normalize input units using aliases
    from_unit_norm = aliases.get(from_unit.lower(), from_unit.lower())
    to_unit_norm = aliases.get(to_unit.lower(), to_unit.lower())

    # Define conversion factors
    conversions = {
        "lbs_to_kg": 0.453592,
        "kg_to_lbs": 2.20462,
        "miles_to_km": 1.60934,
        "km_to_miles": 0.621371
    }

    conversion_key = f"{from_unit_norm}_to_{to_unit_norm}"

    if conversion_key in conversions:
        print(f"{value} * {conversions[conversion_key]}")
        result = value * conversions[conversion_key]
        return {"result": result, "unit": to_unit_norm}
    else:
        return {"error": f"Conversion from {from_unit} to {to_unit} is not supported"}


# Test the function
print(convert_units(56, "miles", "km"))

In [ ]:
# Ask the LLM to use our unit conversion tool
system_prompt = """
You are a helpful assistant that can convert between different units of measurement.
Only respond with the results.

Example responses:

150 pounds is equal to 68 kilograms
68 kilograms is equal to 150 pounds
56 miles is equal to 90 kilometers
90 kilometers is equal to 56 miles

"""

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "How many kilograms is 256 pounds?"}
]

# Call the LLM with our tools
response = completion(model=model, messages=messages, tools=[unit_conversion_tool])
print("First Input:")
print(messages)
print("First Response:")
print(response)
print(response.choices[0].message.content)

# Check if the LLM wants to use a tool
if hasattr(response.choices[0].message, 'tool_calls') and response.choices[0].message.tool_calls:
    tool_calls = response.choices[0].message.tool_calls

    print("\nTool Calls:")

    for tool_call in tool_calls:
        print(f"Tool: {tool_call.function.name}")
        print(f"Arguments: {tool_call.function.arguments}")
        
        # Parse the arguments and call our function
        args = json.loads(tool_call.function.arguments)
        
        if tool_call.function.name == "convert_units":
            result = convert_units(args["value"], args["from_unit"], args["to_unit"])
            print(f"Result: {result}")
            
            # Send the result back to the LLM
            messages.append(response.choices[0].message)
            
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": tool_call.function.name,
                "content": json.dumps(result)
            })
            
            # Get the final response
            final_response = completion(model=model, messages=messages)
            print("\nFinal Input:")
            print(messages)
            print("\nFinal Response:")
            print(final_response.choices[0].message.content)
else:
    print("Model did not call any tools.")